In [1]:
%pylab nbagg
import torch
import numpy as np
from lib.Simulator.Wrapper import *
from lib.utils import *
from numpy import array as ar
from copy import deepcopy as dcp
import numpy as np
from matplotlib import rc
import matplotlib as mpl
import multiprocessing
from multiprocessing import Pool
from lib.Simulator.connectivity_paths import connectivity_source_file
rc('text',usetex = True)
mpl.rcParams['font.size'] = 10
torch.set_printoptions(sci_mode=True)
try:
    multiprocessing.set_start_method('spawn')
except:
    pass
def paral_sim(s, ind,processes = 1):
    if processes > 11:
        processes = 11
    if __name__ == '__main__':
        with Pool(processes=processes) as pool:
            results = pool.map_async(s.run_simulation_long,ind).get(timeout=600000)                
        pool.close()
        pool.join()
    return results
par_val_def = torch.cat([theta_initial.view(1,-1) for i in range(68)], dim = 0)
par_val_def[45,5] = 8
par_val_def[45,-1] = 0
par_val_def[45,-2] = 0
path = f'./Data/heterogeneous_2d_sweep_noise'

Populating the interactive namespace from numpy and matplotlib


In [2]:
batch_par_vals = [[[1e-4*i,10*j,f'Coupled{(i-1)*(10)+(j-1)} (a:{str(1e-4*i)[:6]}, K: {10*j})'] for i in range(k,k+2) for j in range(1,11)] for k in range(1,11,2)]

In [4]:
for batch in range(len(batch_par_vals)):
    results,wrapper,timeseries,psd = None,None,None,None
    par_vals = batch_par_vals[batch]
    wrapper = Wrapper()
    if batch == 0:
        wrapper.add_sim(par_val_def,file = 'empty', a_val = 0, speed = 50,length = 182e3)
    for i in range(len(par_vals)):
        wrapper.add_sim(par_val_def, a_val = par_vals[i][0], speed = par_vals[i][1],length = 182e3) 
    results = paral_sim(wrapper, [i for i in range(len(wrapper.simulations))],processes = len(wrapper.simulations))
    timeseries = np.concatenate([results[i][0].reshape((1,68,-1)) for i in range(len(results))],axis=0)
    psd = torch.cat([results[i][1].view((1,68,-1)) for i in range(len(results))],dim=0)
    np.save(f'{path}{batch}.npy',ar([timeseries,psd,par_val_def, par_vals],dtype = object))

In [5]:
results,wrapper,timeseries,psd = None,None,None,None
#Restart the kernel here before running the rest if the memory is overloaded

In [6]:
data = np.load(f'{path}0.npy', allow_pickle = True)
timeseries, psd, par_val_def, par_vals = data[0], data[1], data[2], ar(data[3], dtype = object)
data = None
for i in range(1,5):
    data = np.load(f'{path}{i}.npy', allow_pickle = True)
    timeseries = np.concatenate((timeseries,data[0]), axis = 0)
    psd = torch.cat((psd,data[1]), dim = 0)
    par_val_def = torch.cat((par_val_def,data[2]), axis = 0)
    par_vals = np.concatenate((par_vals,ar(data[3], dtype = object)), axis = 0)
    data = None

In [7]:
np.save(f'{path}_ts.npy',timeseries)
np.save(f'{path}_global_par.npy',par_vals)
torch.save(psd,f'{path}_psd.pt')
torch.save(par_val_def,f'{path}_local_par.pt')